## Algoritmia
### Práctica 3
El objetivo de esta práctica es trabajar con recurrencias.

Se pide la implementación de las clases y/o funciones que aparecen a continuación.

Las instrucción "pass" que aparecen en el cuerpo de las clases o funciones, se debe sustituir por la implementación adecuada.

Para cada clase o función que se pide se proporciona una o más funciones con algunos tests.

Al llamar a las funciones de test no debería saltar ninguna aserción.

### Función `generador_recurrencia`

In [1]:
def generador_recurrencia(coeficientes, funcion_adicional, iniciales):
    """
    Generador de valores de acuerdo a una recurrencia:
    F(n) = coeficientes[0]*F(n-1) + coeficientes[1]*F(n-2) + ...
         + funcion_adicional(n)
    Los valores iniciales son F(0) = iniciales[0], F(1) = iniciales[1],...
    Los valores que se generan son F(0), F(1), F(2),...
    Se deben generar los valores de uno en uno, no hay que devolver varios.
    Debe generar valores indefinidamente, no hay que poner límites.
    Aunque sea una recurrencia, los valores *no* deben calcularse recursivamente.
    """
    
    pos=0
    ff=[]#Es la lista que contiene los valores iniciales que te dan en el enunciado.
    c=[] #Es la lista que contiene los coeficientes que te dan en el enunciado.
    for i in iniciales:
        ff.append(i)
        pos=pos+1
        yield i
    for j in coeficientes:
        c.append(j)
    
    while True:
        ff.reverse()
        ret=sum(list(map(lambda x, y: x*y, c, ff)))+funcion_adicional(pos)
        ff.reverse()
        ff.append(ret)
        pos=pos+1
        yield ret  

In [2]:
def comprueba_recurrencia(coeficientes, funcion_adicional, iniciales,
                          funcion_alternativa, numero_comprobaciones = 100,
                          epsilon = 0.1):
    """
    Dada una recurrencia (definida en términos de sus coeficientes,
    condiciones inciales y la función_adicional) comprueba si los valores
    generados son (aproximadamente) los mismos que los definidos por una función
    alternativa, para un determinado número de comprobaciones.
    """
    
    iterador = generador_recurrencia(coeficientes, funcion_adicional, iniciales)
    for n in range(numero_comprobaciones):
        #print(n)
        #print("---")
        if abs(next(iterador) - funcion_alternativa(n)) > epsilon:
            return False
    return True

In [3]:
def test_generador_recurrencia():
    """Casos de prueba para la función generador_recurrencia."""

    # Recurrencia f(0)=0, f(n)=f(n-1)+1, que se corresponde con f(n)=n
    assert comprueba_recurrencia([1], lambda n: 1, [0], lambda n: n)
    
    # Recurrencia f(0)=1, f(n)=2*f(n-1), que se corresponde con f(n)=2**n
    assert comprueba_recurrencia([2], lambda n: 0, [1], lambda n: 2**n)
    
    # Recurrencia f(0)=0, f(n)=f(n-1)+n, que se corresponde con f(n)=n*(n+1)/2
    assert comprueba_recurrencia([1], lambda n: n, [0],
                                 lambda n: n * (n + 1) / 2)
    
    # Recurrencia f(0)=0, f(n)=f(n-1)+n/2, que se corresponde con f(n)=n*(n+1)/4
    assert comprueba_recurrencia([1], lambda n: n / 2, [0],
                                 lambda n: n * (n + 1) / 4)
    
    # Recurrencia f(0)=1, f(n)=f(n-1)+2**n, que se corresponde con
    # f(n)=2**(n+1)-1
    assert comprueba_recurrencia([1], lambda n: 2 ** n, [1],
                                 lambda n: 2 ** (n + 1) - 1)
    
    # Recurrencia f(0)=0, f(1)=1, f(n)=4f(n-1)-4f(n-2), que se corresponde con
    # f(n)=2**(n-1)*n
    assert comprueba_recurrencia([4, -4], lambda n: 0, [0, 1],
                                 lambda n: 2 ** (n - 1) * n)
    
    # Recurrencia f(0)=0, f(1)=1, f(n)=2f(n-1)-f(n-2)+1, que se corresponde con
    # f(n)=n*(n+1)/2
    assert comprueba_recurrencia([2, -1], lambda n: 1, [0, 1],
                                 lambda n: n * (n + 1) / 2)
    
    # Recurrencia f(0)=0, f(1)=1, f(2)=2, f(n)=f(n-1)+f(n-2)-f(n-3), que se
    # corresponde con f(n)=n
    assert comprueba_recurrencia([1, 1, -1],lambda n: 0, [0, 1, 2],
                                 lambda n: n)
    

if __name__ == "__main__": 
    test_generador_recurrencia()
    print("OK")    

OK


### Clase `RecurrenciaMaestra`

In [4]:
from math import log

class RecurrenciaMaestra: 
    """
    Clase que representa una recurrencia de las que se consideran en el 
    teorema maestro, de la forma T(n)=aT(n/b)+n^k. Se interpreta que en n/b
    la división es entera.
    Además de los métodos que aparecen a continuación, tienen que funcionar 
    los siguientes operadores: 
        ==, !=,
        str(): la representación como cadena debe ser 'aT(n/b)+n^k'
        []: el parámetro entre corchetes es el valor de n para calcular T(n).
    """
    
    def __init__(self, a, b, k, inicial = 0):
        """
        Constructor de la clase, los parámetros a, b, y k son los que
        aparecen en la fórmula aT(n/b)+n^k. El parámetro inicial es el valor
        para T(0).
        """
        self._a=a
        self._b=b
        self._k=k
        self._inicial=inicial
        self._l={}
        self._l.setdefault(0,self._inicial) 
    def metodo_maestro(self):
        """
        Devuelve una cadena con el tiempo de la recurrencia de acuerdo al 
        método maestro. La salida está en el formato "O(n^x)" o "O(n^x*log(n))",
        siendo x un número.
        """
        bk= self._b**self._k
        if self._a < bk:
            ret="O(n^"+str(self._k)+")"
        elif self._a is bk:
            ret="O(n^"+str(self._k)+"*log(n))"
        elif self._a > bk:
            loga=log(self._a)/log(self._b)
            ret="O(n^"+str(loga)+")"
        return ret
       
    def __iter__(self):
        """
        Generador de valores de la recurrencia: T(0), T(1), T(2), T(3)..., 
        indefinidamente.
        Aunque sea una recurrencia, los valores *no* deben calcularse 
        recursivamente.
        """
        n=0
        while True:
            nb=int(n/self._b)
            if self._k is 0 and n is 0:
                ret=self._a*self._l.get(nb)
            else:
                ret=self._a*self._l.get(nb)+n**self._k 
                
            self._l.setdefault(n,ret)
            n=n+1
            yield ret
            

    def __eq__(self, other):
        if self._a is other._a and self._b is other._b and self._k is other._k:
            return True
        else:
            return False
        
    def __str__(self):
        "2T(n/2)+n^2"
        ret=str(self._a)+"T(n/"+str(self._b)+")+n^"+str(self._k)
        return ret
        
    def __getitem__(self,n):
        nb=int(n/self._b)
        nk=n**self._k
        if n is 0 and self._k is 0:
            nk=0
        if n is not 0:
            ret=self._a*self[nb]+nk
        else:
            ret=self._a*self._l.get(0)+nk
        return ret

In [5]:
def test_recurrencia_maestra_metodo_maestro(): 
    """Casos de prueba para RecurrenciaMaestra.metodo_maestro().""" 
    
    # Recurrencia T(n)=3T(n/2)+O(n^2)
    resultado = RecurrenciaMaestra(3, 2, 2).metodo_maestro()
    assert resultado == "O(n^2)"

    # Recurrencia T(n)=2T(n/2)+O(n)
    resultado = RecurrenciaMaestra(2, 2, 1).metodo_maestro()
    assert resultado == "O(n^1*log(n))"

    # Recurrencia T(n)=3T(n/2)+O(n)
    resultado = RecurrenciaMaestra(3, 2, 1).metodo_maestro()
    # esperamos algo parecido a "O(n^1.5849625007211563)"
    assert "O(n^1.58" in resultado
    assert "log" not in resultado    
                 
if __name__ == "__main__":
    test_recurrencia_maestra_metodo_maestro()
    print("OK")

OK


In [6]:
def test_recurrencia_maestra_operadores(): 
    """Casos de prueba para los operadores de RecurrenciaMaestra."""
    
    r1 = RecurrenciaMaestra(2, 2, 2)
    # Tests para los operadores == y !=
    assert r1 == RecurrenciaMaestra(2, 2, 2),r1
    assert not r1 != RecurrenciaMaestra(2, 2, 2)
    
    for a, b, k in ((1, 1, 1), (1, 1, 2), (1, 2, 1), (2, 1, 1) ):
        assert r1 != RecurrenciaMaestra(a, b, k)
        assert not r1 == RecurrenciaMaestra(a, b, k)
    
    # Tests para str()
    assert str(r1) == "2T(n/2)+n^2"
    assert str(RecurrenciaMaestra(7, 4, 3)) == "7T(n/4)+n^3"
    
    # Tests para []
    for n, valor in enumerate((0, 1, 6, 11, 28, 37, 58, 71, 120, 137, 174, 195, 
                               260, 285, 338, 367, 496, 529, 598, 635)):
        assert r1[n] == valor,r1[n]
        
    r2 = RecurrenciaMaestra(1, 2, 0, 1) 
    for n, valor in enumerate((1, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 
                               6, 6, 6, 6)):
    
            assert r2[n] == valor, r2[n]
       
               
    r3 = RecurrenciaMaestra(4, 3, 1)
    for n, valor in enumerate((0, 1, 2, 7, 8, 9, 14, 15, 16, 37, 38, 39, 44, 45,
                               46, 51, 52, 53, 74, 75)):
        
        assert r3[n] == valor, r3[n]  
    
    
if __name__ == "__main__":
    test_recurrencia_maestra_operadores()
    print("OK")

OK


In [7]:
def test_recurrencia_maestra_genera(): 
    """Casos de prueba para la iteración sobre RecurrenciaMaestra."""
    
    for v1, v2 in zip(RecurrenciaMaestra(2, 2, 2),
                      (0, 1, 6, 11, 28, 37, 58, 71, 120, 137, 174, 195, 260, 
                       285, 338, 367, 496, 529, 598, 635)):
        assert v1 == v2

    for v1, v2 in zip(RecurrenciaMaestra(1, 2, 0, 1),
                      (1, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6,
                       6)):
        assert v1 == v2
        
    for v1, v2 in zip(RecurrenciaMaestra(4, 3, 1),
                      (0, 1, 2, 7, 8, 9, 14, 15, 16, 37, 38, 39, 44, 45, 46, 51,
                       52, 53, 74, 75)):
        assert v1 == v2        

        
if __name__ == "__main__":
    test_recurrencia_maestra_genera()
    print("OK")

OK
